<a href="https://colab.research.google.com/github/8dci/T5/blob/main/Test_of_Question_Answering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning Question Answering Model

This exam will guide you through loading, preprocessing, and fine-tuning a pre-trained model for a question-answering task using a dataset. Follow the steps carefully.

### Model and Dataset Information

For this task, you will be working with the following:

- **Model Checkpoint**: Use the pre-trained model checkpoint `distilbert-base-cased` for both the model and tokenizer.
- **Dataset**: You will be using the `christti/squad-augmented-v2` dataset. Ensure to load and preprocess the dataset correctly for training and evaluation.

**Note:**
- Any additional steps or methods you include that improve or enhance the results will be rewarded with bonus points if they are justified.
- The steps outlined here are suggestions. You are free to implement alternative methods or approaches to achieve the task, as long as you explain the reasoning and the process at the bottom of the notebook.
- You can use either TensorFlow or PyTorch for this task. If you prefer TensorFlow, feel free to use it when working with Hugging Face Transformers.
- The number of data samples you choose to work with is flexible. However, if you select a very low number of samples and the training time is too short, this could affect the evaluation of your work.

## Step 1: Load the Dataset

Load the dataset and split it into training and test sets. Use 20% of the data for testing.

In [ ]:
!pip install datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset('christti/squad-augmented-v2',split=['train[:20%]'])

In [ ]:
train_dataset = dataset[0].train_test_split(test_size=0.2)

train_data = train_dataset['train']
test_data = train_dataset['test']

print(len(train_data))
len(test_data)

27073


6769

In [ ]:
print(type(train_data))

<class 'datasets.arrow_dataset.Dataset'>


In [ ]:
train_data

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 27073
})

## Step 2: Load the Pretrained Tokenizer and Model

Use the model and tokenizer for the question-answering task.

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-cased')

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Step 3: Preprocess the Dataset

Define a function to preprocess the dataset by tokenizing both the context and the question. The function will also calculate the start and end positions of the answers. In the tokenizer you might face a problem if you use `truncation=True` so consider using `truncation='only_first'` if needed.

In [ ]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
train_data = train_data.map(preprocess_function,batched=True,remove_columns=['id', 'title', 'context', 'question', 'answers'])
test_data = test_data.map(preprocess_function,batched=True,remove_columns=['id', 'title', 'context', 'question', 'answers'])

Map:   0%|          | 0/27073 [00:00<?, ? examples/s]

Map:   0%|          | 0/6769 [00:00<?, ? examples/s]

## Step 4: Define Training Arguments and Initialize the Trainer

Set up the training configuration with parameters like learning rate, batch size, and number of epochs.

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
training_args = TrainingArguments(output_dir='./outputs',learning_rate=1e-5 , per_device_train_batch_size=32, num_train_epochs=1)
trainer = Trainer(model=model, args=training_args, train_dataset=train_data)

## Step 5: Fine-tune the Model

Run the training process using the initialized trainer.

In [ ]:
trainer.train()

Step,Training Loss
500,3.217800


TrainOutput(global_step=847, training_loss=2.8242281178516087, metrics={'train_runtime': 902.5563, 'train_samples_per_second': 29.996, 'train_steps_per_second': 0.938, 'total_flos': 2652878532874752.0, 'train_loss': 2.8242281178516087, 'epoch': 1.0})

## Step 6: Inference

Once the model is trained, perform inference by answering a question based on a context. Use the tokenizer to process the input, and then feed it into the model to get the predicted answer.

In [ ]:
metrics = trainer.evaluate(test_data)


In [ ]:
print(metrics)

{'eval_loss': 2.070546865463257, 'eval_runtime': 84.8862, 'eval_samples_per_second': 79.742, 'eval_steps_per_second': 9.978, 'epoch': 1.0}


In [ ]:
context = """
The Apollo program, also known as Project Apollo, was the third United States human spaceflight program carried out by the
National Aeronautics and Space Administration (NASA), which accomplished landing the first humans on the Moon from
"""

question = "What was the goal of the Apollo program?"

def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        [examples["context"]] * len(questions),
        padding="max_length",
        max_length=384,
        truncation=True
    )
    return inputs

test_smaple = preprocess_function({'context':context,'question':question})
test_dataset = Dataset.from_dict(test_smaple)

predictions = trainer.predict(test_dataset)

In [ ]:
print(predictions)

PredictionOutput(predictions=(array([[-1.0693061 , -0.38122153, -4.7462015 , ..., -6.7726283 ,
        -6.7501564 , -6.3868184 ],
       [-1.0638636 ,  0.13764754, -4.754094  , ..., -6.818981  ,
        -6.7946277 , -6.43981   ],
       [-1.0271146 ,  0.1661433 , -4.733578  , ..., -6.735037  ,
        -6.7136045 , -6.3128242 ],
       ...,
       [-1.0271146 ,  0.1661433 , -4.733578  , ..., -6.735037  ,
        -6.7136045 , -6.3128242 ],
       [-1.0606842 ,  0.3325307 , -4.745342  , ..., -6.786084  ,
        -6.75904   , -6.3661275 ],
       [-1.4178772 , -1.5220299 , -4.8133917 , ..., -6.976784  ,
        -6.965768  , -6.6104918 ]], dtype=float32), array([[-1.4011167, -2.111544 , -3.6071725, ..., -4.3679476, -4.4216137,
        -4.86906  ],
       [-1.4013814, -1.052557 , -3.5964584, ..., -4.414474 , -4.469485 ,
        -4.897627 ],
       [-1.406949 , -1.1203246, -3.6225262, ..., -4.505862 , -4.5002275,
        -4.9153767],
       ...,
       [-1.406949 , -1.1203246, -3.6225262, ...

In [ ]:
start_logits = predictions.predictions[0]
end_logits = predictions.predictions[1]
predicted_start_indexes = start_logits.argmax(axis=-1)
predicted_end_indexes = end_logits.argmax(axis=-1)

In [ ]:
start_idx = predicted_start_indexes[0]
end_idx = predicted_end_indexes[0]
answer = tokenizer.decode(test_smaple['input_ids'][i][start_idx : end_idx+1])

In [ ]:
print(answer)

National Aeronautics and Space Administration
